In [1]:
import numpy as np
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data

Using TensorFlow backend.


In [2]:
def get_classes(classes_path):
    '''loads the classes'''
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

In [3]:
def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

In [13]:
annotation_path = 'row_format.txt'
log_dir = 'model_data/'
classes_path = 'model_data/hand.txt'
anchors_path = 'model_data/yolo_anchors.txt'
class_names = get_classes(classes_path)
num_classes = len(class_names)
anchors = get_anchors(anchors_path)
input_shape = (416, 416)

In [14]:
anchors

array([[  10.,   13.],
       [  16.,   30.],
       [  33.,   23.],
       [  30.,   61.],
       [  62.,   45.],
       [  59.,  119.],
       [ 116.,   90.],
       [ 156.,  198.],
       [ 373.,  326.]])

In [ ]:
is_tiny_version = len(anchors)==6
if is_tiny_version:
    model = create_tiny_model(input_shape, anchors)

In [6]:
def get_random_data(annotation_line, input_shape, random=True,
                   max_boxes=20, jitter=.3, hue=.1,
                   sat=1.5, val=1.5, proc_img=True):
    line = annotation_line.split()
    image = Image.open(line[0])
    iw, ih = image.size
    h, w = input_shape
    box = np.array([np.array(list(max(int, box.split(',')))) for box in line[1:]])

In [7]:
def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            if i == 0:
                np.random.shuffle(annotation_lines)
            image, box = get_random_data(annotation_lines[i], input_shape, random=True)